In [1]:
!pip install --upgrade pip
!pip install --disable-pip-version-check torch torchdata --quiet
!pip install transformers datasets evaluate rouge_score loralib peft --quiet
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0
    Uninstalling pip-25.0:
      Successfully uninstalled pip-25.0
  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [53]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import pandas as pd
from datasets import Dataset

In [ ]:
!wget https://groups.inf.ed.ac.uk/ami/AMICorpusAnnotations/ami_public_manual_1.6.2.zip
!unzip ami_public_manual_1.6.2.zip

In [12]:
# AMI Summarization Dataset Preparation Script (Updated for multiple speaker word files)
import os
import xml.etree.ElementTree as ET
import pandas as pd

# Parse words file to extract and reconstruct transcript
def parse_words_file(path):
    tree = ET.parse(path)
    root = tree.getroot()
    words = []
    for w in root.findall(".//w"):
        start_time = float(w.attrib.get("starttime", 0.0))
        end_time = float(w.attrib.get("endtime", 0.0))
        text = w.text or ""
        words.append((start_time, end_time, text))
    return words

def build_transcript(words):
    words.sort(key=lambda x: x[0])
    return " ".join(word for _, _, word in words)

# Parse summary XML file to extract human-written abstract, actions, and decisions
def parse_summary_xml(path):
    tree = ET.parse(path)
    root = tree.getroot()

    def extract_sentences(section):
        if section is None:
            return ""
        return " ".join([s.text for s in section.findall("sentence") if s.text and s.text.strip() != "*NA*"])

    abstract = extract_sentences(root.find(".//abstract"))
    actions = extract_sentences(root.find(".//actions"))
    decisions = extract_sentences(root.find(".//decisions"))
    combined_summary = " ".join([abstract, actions, decisions])
    return combined_summary

# Process all meetings in AMI dataset
def process_all_ami_meetings(base_path):
    words_dir = os.path.join(base_path, "words")
    abssumm_dir = os.path.join(base_path, "abstractive")

    dataset = []

    for fname in os.listdir(abssumm_dir):
        if not fname.endswith(".xml"):
            continue

        meeting_id = fname.split(".")[0]  # e.g., TS3010b
        summary_path = os.path.join(abssumm_dir, fname)

        # Collect all speaker word files: TS3010b.A.words.xml, TS3010b.B.words.xml, etc.
        meeting_word_files = [
            os.path.join(words_dir, f)
            for f in os.listdir(words_dir)
            if f.startswith(meeting_id + ".") and f.endswith(".words.xml")
        ]

        if not meeting_word_files:
            continue

        try:
            # Combine transcripts from all channels
            full_word_list = []
            for wp in meeting_word_files:
                full_word_list.extend(parse_words_file(wp))
            transcript = build_transcript(full_word_list)

            # Parse summary
            summary = parse_summary_xml(summary_path)

            # Add to dataset
            dataset.append({
                "meeting_id": meeting_id,
                "input_text": transcript[:3000],  # Trim for practicality
                "target_summary": summary
            })
        except Exception as e:
            print(f"Error processing {meeting_id}: {e}")
            continue

    return pd.DataFrame(dataset)

ami_base_dir = ""  # Update this path
summarization_df = process_all_ami_meetings(ami_base_dir)
summarization_df.to_csv("ami_summarization_dataset.csv", index=False)
print("Saved summarization dataset with", len(summarization_df), "entries.")


Saved summarization dataset with 142 entries.


In [13]:
# Load manually with pandas
df = pd.read_csv("ami_summarization_dataset.csv")

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

In [51]:
import getpass
from huggingface_hub import login

token = getpass.getpass("Enter your Hugging Face token: ")
login(token=token)

Enter your Hugging Face token:  ········


In [57]:
model_name='mistralai/Mistral-7B-v0.1'

model = AutoModelForCausalLM.from_pretrained(model_name,
torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [58]:
def tokenize_function(example):
    start_prompt = 'Summarize this meeting transcript in 1-2 factual sentences:.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["input_text"]]

    # Tokenize inputs
    inputs = tokenizer(prompt, padding="max_length", truncation=True, max_length=256)

    # Tokenize targets (labels)
    labels = tokenizer(example["target_summary"], padding="max_length", truncation=True, max_length=256)

    # Replace padding token id's in labels with -100 so they are ignored in loss
    labels["input_ids"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels["input_ids"]
    ]

    # Combine into a single return dict
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply to both splits
tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)

# Remove original columns from each
columns_to_remove = dataset["train"].column_names
tokenized_train = tokenized_train.remove_columns(columns_to_remove)
tokenized_test = tokenized_test.remove_columns(columns_to_remove)

Map:   0%|          | 0/127 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [59]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_train.shape}")
print(f"Test: {tokenized_test.shape}")
print(tokenized_train)

Shapes of the datasets:
Training: (127, 3)
Test: (15, 3)
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 127
})


In [60]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


In [61]:
print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 7241732096
all model parameters: 7241732096
percentage of trainable model parameters: 100.00%


In [62]:
def generate_summary(transcript: str, max_new_tokens: int = 80) -> str:
    prompt = f"Summarize this meeting transcript in 1-2 factual sentences:. Transcript:{transcript.strip()}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    print(prompt)

    with torch.no_grad():
        output = peft_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3,
            repetition_penalty=1.5,
            length_penalty=1.0
        )

    return tokenizer.decode(output[0], skip_special_tokens=True).split("Summary:")[-1].strip()

import re
def truncate_after_n_sentences(text, n=4):
    """
    Truncate a string after n full sentences. Keeps punctuation.
    """
    # Matches sentences ending with ., !, or ?, optionally followed by quotes or whitespace
    sentence_endings = re.findall(r'[^.!?]*[.!?]["\']?\s*', text)

    # Take the first n non-empty sentences
    non_empty = [s.strip() for s in sentence_endings if s.strip()]
    truncated = ' '.join(non_empty[:n])
    return truncated

## Configuration - 1 (Mistral 7B fine-tuning)

In [ ]:
model_name='mistralai/Mistral-7B-v0.1'

model = AutoModelForCausalLM.from_pretrained(model_name,
torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
def tokenize_function(example):
    start_prompt = 'Summarize this meeting transcript in 1-2 factual sentences:.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["input_text"]]

    # Tokenize inputs
    inputs = tokenizer(prompt, padding="max_length", truncation=True, max_length=256)

    # Tokenize targets (labels)
    labels = tokenizer(example["target_summary"], padding="max_length", truncation=True, max_length=256)

    # Replace padding token id's in labels with -100 so they are ignored in loss
    labels["input_ids"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels["input_ids"]
    ]

    # Combine into a single return dict
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply to both splits
tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)

# Remove original columns from each
columns_to_remove = dataset["train"].column_names
tokenized_train = tokenized_train.remove_columns(columns_to_remove)
tokenized_test = tokenized_test.remove_columns(columns_to_remove)

In [65]:
output_dir = f'./meeting-summary-training_mistral'
training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-5,
    num_train_epochs=5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    )

In [66]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,5.995100,6.055297
2,5.872700,6.006083
3,5.339100,6.050463
4,4.218800,6.802176
5,2.965600,7.453739


TrainOutput(global_step=320, training_loss=4.890133386850357, metrics={'train_runtime': 742.8466, 'train_samples_per_second': 0.855, 'train_steps_per_second': 0.431, 'total_flos': 6935453431234560.0, 'train_loss': 4.890133386850357, 'epoch': 5.0})

In [67]:
# === Example Usage === #

test_transcript = """
Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.
    """

summary = generate_summary(test_transcript)
print("\n=== Generated Summary ===\n")
print(truncate_after_n_sentences(summary))

Summarize this meeting transcript in 1-2 factual sentences:. Transcript:Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.

=== Generated Summary ===

m  N���� reflection certain�������ton����able Bar mY ter��op�������itiesAL? ector	irst m,M. .


## Configuration - 2 (Flan-T5-Large fine-tuning)

In [72]:
model_name='google/flan-t5-large'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name,
torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [73]:
def tokenize_function(example):
    start_prompt = 'Summarize this meeting transcript in 1-2 factual sentences:.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["input_text"]]

    # Tokenize inputs
    inputs = tokenizer(prompt, padding="max_length", truncation=True, max_length=256)

    # Tokenize targets (labels)
    labels = tokenizer(example["target_summary"], padding="max_length", truncation=True, max_length=256)

    # Replace padding token id's in labels with -100 so they are ignored in loss
    labels["input_ids"] = [
        [(token if token != tokenizer.pad_token_id else -100) for token in label]
        for label in labels["input_ids"]
    ]

    # Combine into a single return dict
    inputs["labels"] = labels["input_ids"]
    return inputs

# Apply to both splits
tokenized_train = dataset["train"].map(tokenize_function, batched=True)
tokenized_test = dataset["test"].map(tokenize_function, batched=True)

# Remove original columns from each
columns_to_remove = dataset["train"].column_names
tokenized_train = tokenized_train.remove_columns(columns_to_remove)
tokenized_test = tokenized_test.remove_columns(columns_to_remove)

Map:   0%|          | 0/127 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [74]:
output_dir = f'./meeting-summary-training_flan'
training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-5,
    num_train_epochs=5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    )

In [75]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.318800,3.221875
2,3.476600,3.206250
3,3.273400,3.198958
4,3.423400,3.198958
5,3.403100,3.198958


TrainOutput(global_step=320, training_loss=3.38037109375, metrics={'train_runtime': 98.7005, 'train_samples_per_second': 6.434, 'train_steps_per_second': 3.242, 'total_flos': 731763824394240.0, 'train_loss': 3.38037109375, 'epoch': 5.0})

In [76]:
# === Example Usage === #

test_transcript = """
Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.
    """

summary = generate_summary(test_transcript)
print("\n=== Generated Summary ===\n")
print(truncate_after_n_sentences(summary))

Summarize this meeting transcript in 1-2 factual sentences:. Transcript:Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.

=== Generated Summary ===

The project manager opened the meeting by stating that the team had just finished writing the dockerfiles for the CI/CD pipeline and that they needed to add Kubernetes, Prometheus, Grafana, and MLFlow to the overall pipeline. The team members then discussed their roles in the project and what they would like to accomplish in the next meeting.


## Configuration - 3 (Flan-T5-Large with LoRA with lr 1e-5)


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType
lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [ ]:
from peft import PeftModel

if isinstance(peft_model, PeftModel):
    peft_model = peft_model.unload()

peft_model = get_peft_model(model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

In [21]:
output_dir = f'./meeting-summary-training2'
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-4,
    num_train_epochs=5,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    )


peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    )

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
peft_trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,3.245300,3.093750
2,3.268700,3.001042
3,2.967200,2.947917
4,3.064100,2.920833
5,3.068800,2.911458


TrainOutput(global_step=320, training_loss=3.146435546875, metrics={'train_runtime': 76.1405, 'train_samples_per_second': 8.34, 'train_steps_per_second': 4.203, 'total_flos': 734064987340800.0, 'train_loss': 3.146435546875, 'epoch': 5.0})

In [25]:
# === Example Usage === #

test_transcript = """
Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.
    """

summary = generate_summary(test_transcript)
print("\n=== Generated Summary ===\n")
print(truncate_after_n_sentences(summary))

Summarize this meeting transcript in 1-2 factual sentences:. Transcript:Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.

=== Generated Summary ===

We discussed the CI/CD pipeline, Prometheus and Grafana dashboards, Kubernetes, model training and MLFlow implementation.


## Configuration - 4 (Flan-T5-Large with LoRA with lr 1e-3)

In [30]:
from peft import LoraConfig, get_peft_model, TaskType
lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [33]:
from peft import PeftModel

if isinstance(peft_model, PeftModel):
    peft_model = peft_model.unload()

peft_model = get_peft_model(model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 2359296
all model parameters: 785509376
percentage of trainable model parameters: 0.30%


In [34]:
output_dir = f'./meeting-summary-training3'
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-3,
    num_train_epochs=5,
    logging_steps=10,
    eval_strategy="epoch",     
    save_strategy="epoch",
    report_to="none",
    )


peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    )

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [35]:
peft_trainer.train()

Epoch,Training Loss,Validation Loss
1,2.876600,2.720833
2,2.740600,2.595833
3,2.376600,2.517708
4,2.368000,2.477083
5,2.414100,2.467708


TrainOutput(global_step=320, training_loss=2.59228515625, metrics={'train_runtime': 76.1553, 'train_samples_per_second': 8.338, 'train_steps_per_second': 4.202, 'total_flos': 734064987340800.0, 'train_loss': 2.59228515625, 'epoch': 5.0})

In [36]:
# === Example Usage === #

test_transcript = """
Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.
    """

summary = generate_summary(test_transcript)
print("\n=== Generated Summary ===\n")
print(truncate_after_n_sentences(summary))

Summarize this meeting transcript in 1-2 factual sentences:. Transcript:Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.

=== Generated Summary ===

The project manager opened the meeting by introducing himself to the group and stating the agenda for the next meeting. The industrial designer presented his prototype of a remote control device that can be used to control a television. The marketing expert discussed the current trends in remote control devices and discussed how to incorporate them into the design of the remote.


## Configuration - 5 (Flan-T5-Large with LoRA with lr 1e-4)

In [77]:
from peft import LoraConfig, get_peft_model, TaskType
lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

In [78]:
from peft import PeftModel

if isinstance(peft_model, PeftModel):
    peft_model = peft_model.unload()

peft_model = get_peft_model(model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 2359296
all model parameters: 785509376
percentage of trainable model parameters: 0.30%


In [79]:
output_dir = f'./meeting-summary-training4'
peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-4,
    num_train_epochs=5,
    logging_steps=10,
    eval_strategy="epoch",      
    save_strategy="epoch",
    report_to="none",
    )


peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    )

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [80]:
peft_trainer.train()

Epoch,Training Loss,Validation Loss
1,3.223400,3.073958
2,3.245300,2.985417
3,2.945300,2.932292
4,3.046900,2.905208
5,3.050000,2.900000


TrainOutput(global_step=320, training_loss=3.122119140625, metrics={'train_runtime': 77.9665, 'train_samples_per_second': 8.145, 'train_steps_per_second': 4.104, 'total_flos': 734064987340800.0, 'train_loss': 3.122119140625, 'epoch': 5.0})

In [82]:
# === Example Usage === #

test_transcript = """
Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.
    """

summary = generate_summary(test_transcript)
print("\n=== Generated Summary ===\n")
print(truncate_after_n_sentences(summary))

Summarize this meeting transcript in 1-2 factual sentences:. Transcript:Hi everyone! I just finished writing the dockerfiles for the CI/CD pipeline. What about you? I have built the Prometheus and Grafana dashboards. We now need to integrate the same into our overall pipeline. Yeah, we also need to add Kubernetes to it. What about the model training and MLFlow implementation? I've started working on it, will also add Ray Tune. Cool, let us meet later.

=== Generated Summary ===

We discussed the CI/CD pipeline, Prometheus and Grafana dashboards, Kubernetes, model training and MLFlow implementation.
